# I. Set-up

In [11]:
# Standard imports
import pandas as pd
import numpy as np
import json

# Geospatial processing packages
from matplotlib import pyplot as plt
from shapely import wkt
import geopandas as gpd
import geopandas as gpd
import rasterio  # as rio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling

# import geemap
# import geemap.eefolium as emap

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl

# II. Extracting NOAH Hazard Level Data

## A. Setting up Administrative Boundary Data

In [ ]:
# Input and Output Data Directory
INPUT_DIR = "/path/to/your/data"
OUTPUT_DIR = "/path/to/your/output"

In [12]:
# Read Lacuna Administrative Boundary Data
adm_fn = f"{INPUT_DIR}/target_admin_boundaries/target_admin_bounds.shp"
adm_gdf = gpd.read_file(adm_fn).to_crs("EPSG:32651")

# Apply Geometry Fix
adm_gdf["geometry"] = adm_gdf["geometry"].apply(
    lambda x: wkt.loads(wkt.dumps(x, rounding_precision=4))
)

# Compute Area
adm_gdf["brgy_total_area_sqkm"] = adm_gdf.geometry.area.div(1000000).round(4)
adm_gdf.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_total_area_sqkm
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((214045.003 1776779.832, 214019.452 1...",1.0216
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((214708.019 1775166.057, 214717.717 1...",1.0440
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((216189.272 1777401.028, 216215.228 1...",3.2625
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((215436.528 1775726.907, 215437.143 1...",0.1814
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((298678.543 1723395.559, 299145.784 1...",6.3369


In [2]:
lacuna_provinces = [
    "Albay",
    "Cebu",
    "DavaoDelSur",
    "Iloilo",
    "Leyte",
    "MetroManila",
    "MetroManila",
    "MetroManila",
    "MisamisOriental",
    "Pangasinan",
    "NuevaEcija",
    "ZamboangaDelSur",
]

In [14]:
lacuna_cities = [
    "Legazpi City",
    "Mandaue City",
    "Davao City",
    "Iloilo City",
    "Tacloban City",
    "City of Muntinlupa",
    "City of Mandaluyong",
    "City of Navotas",
    "Cagayan de Oro City",
    "Dagupan City",
    "Palayan City",
    "Zamboanga City",
]

## B. 100-year Flood Hazard 

### 1. Extracting Hazard Level Percent Area

Compute the percent area of each 25-year flood hazard level per baranggay across the Lacuna Cities. 


In [15]:
# Setup an empty list that will eventually contain all the Lacuna City Dataframes with ammended Flood Hazard Area Data
city_gdf_list = []

# Target Output Columns
pivot_city_column_names_100yr_fl = [
    "brgy_pct_area_flood_hazard_100yr_low",
    "brgy_pct_area_flood_hazard_100yr_med",
    "brgy_pct_area_flood_hazard_100yr_high",
]

# Create a loop across all Lacuna Cities
for count, province in enumerate(lacuna_provinces):
    print(count, province, lacuna_cities[count])

    # Select Lacuna City <i> from Administrative Boundary Data
    city_gdf = adm_gdf[adm_gdf["ADM3_EN"] == lacuna_cities[count]]

    # Read Provincial 100-Year Flood Hazard Data
    fl_hzrd_fn = f"{INPUT_DIR}/NOAH-Flood-Hazard/100-year/{province}_Flood_100year.shp"
    fl_hzrd_gdf = gpd.read_file(fl_hzrd_fn).to_crs("EPSG:32651")

    # Clip Flood Hazard Data with Lacuna City <i>'s boundary.
    clipped_fl_hzrd_gdf = gpd.clip(fl_hzrd_gdf, city_gdf)

    # Get Intersection of each Hazard with each Baranggay
    overlay_city_fl_hzrd_gdf = gpd.overlay(
        city_gdf, clipped_fl_hzrd_gdf, how="intersection", keep_geom_type=False
    )

    # Calculate the area (inkm2) of the resulting intersecting polygons
    overlay_city_fl_hzrd_gdf["area"] = overlay_city_fl_hzrd_gdf.geometry.area.div(
        1000000
    )

    # Aggregate the area of the intersecting polygons by Baranggay using a Pivot Table
    pivot_city_gdf = overlay_city_fl_hzrd_gdf.pivot_table(
        index="ADM4_EN", columns="Var", values="area", aggfunc=np.sum
    )

    # Rename Columns for Readability. Note that the values are still in km2.
    pivot_city_gdf.columns = pivot_city_column_names_100yr_fl

    # display(pivot_city_gdf)

    # Merge the computed pivot table data to the original city administrative data
    city_gdf = city_gdf.merge(pivot_city_gdf, on="ADM4_EN", how="left")

    # Compute Percent Area out of Baranggay Area
    city_gdf[pivot_city_column_names_100yr_fl] = (
        city_gdf[pivot_city_column_names_100yr_fl].div(
            city_gdf["brgy_total_area_sqkm"], axis=0
        )
        * 100
    )
    # display(city_gdf.iloc[:,10:])

    # Add city data to list of city dataframes
    city_gdf_list.append(city_gdf)

# Concatenate Lists of Lacuna City with Hazard Data Dataframes
fl_100_year_gdf = pd.concat(city_gdf_list)
fl_100_year_gdf.head()

0 Albay Legazpi City


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_total_area_sqkm,brgy_pct_area_flood_hazard_100yr_low,brgy_pct_area_flood_hazard_100yr_med,brgy_pct_area_flood_hazard_100yr_high
0,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 45 - Dita (Bgy. 51),PH050506053,"POLYGON ((579752.343 1458208.276, 580219.811 1...",0.8428,18.770255,0.688449,0.090723
1,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 62 - Homapon (Bgy. 55),PH050506056,"POLYGON ((582204.624 1447546.937, 582206.580 1...",7.0818,3.306187,5.833723,6.514679
2,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 10 - Cabugao,PH050506072,"POLYGON ((579556.773 1452590.900, 579560.114 1...",0.0335,4.426383,1.878421,NaN
3,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 23 - Imperial Court Subd. (Pob.),PH050506020,"POLYGON ((581731.582 1452914.659, 581758.664 1...",0.0621,7.036058,68.688661,22.837934
4,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 40 - Cruzada (Bgy. 52),PH050506051,"POLYGON ((580282.649 1455133.588, 580380.407 1...",1.4442,17.295738,5.928618,0.118485


### 2. Plotting Flood Hazard Maps

In [ ]:
# Create a loop across all Lacuna Cities
for count, province in enumerate(lacuna_provinces):
    # print(count,province, lacuna_cities[count])

    # Select Lacuna City <i> from Administrative Boundary Data
    city_gdf = adm_gdf[adm_gdf["ADM3_EN"] == lacuna_cities[count]]

    # Read Provincial 100-Year Flood Hazard Data
    fl_hzrd_fn = f"{INPUT_DIR}/FloodHazard/100-year/{province}_Flood_100year.shp"
    fl_hzrd_gdf = gpd.read_file(fl_hzrd_fn).to_crs("EPSG:32651")

    # Clip Flood Hazard Data with Lacuna City <i>'s boundary.
    clipped_fl_hzrd_gdf = gpd.clip(fl_hzrd_gdf, city_gdf)

    # Create the figure and axes
    fig, ax = plt.subplots()

    # Plot the Administrative Boundary Basemap (first layer)
    city_gdf.plot(ax=ax, facecolor="none", edgecolor="gray")

    # Crop the visualization to the extent of the first layer
    ax.set_xlim(city_gdf.total_bounds[0], city_gdf.total_bounds[2])
    ax.set_ylim(city_gdf.total_bounds[1], city_gdf.total_bounds[3])
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    # Plot the second layer (flood hazard)
    clipped_fl_hzrd_gdf.plot(
        ax=ax,
        column="Var",
        categorical=True,
        cmap="YlOrRd",
        alpha=0.7,
        legend=True,
        legend_kwds={"loc": "upper center", "bbox_to_anchor": (0.5, -0.05), "ncol": 3},
    )

    # Set Title
    plt.title("100-Year Flood Hazard Map of " + lacuna_cities[count])

    # Save the figure
    # plt.savefig(f'{OUTPUT_DIR}/Plots/{lacuna_cities[count]}_Flood_100_year.png')

    # Show the plot
    plt.show()

## C. 25-year Flood Hazard

### 1. Extracting Hazard Level Percent Area

Compute the percent area of each 25-year flood hazard level per baranggay across the Lacuna Cities. 

In [16]:
# Setup an empty list that will eventually contain all the Lacuna City Dataframes with ammended Flood Hazard Area Data
city_gdf_list = []

# Target Output Columns
pivot_city_column_names_25yr_fl = [
    "brgy_pct_area_flood_hazard_25yr_low",
    "brgy_pct_area_flood_hazard_25yr_med",
    "brgy_pct_area_flood_hazard_25yr_high",
]

# Create a loop across all Lacuna Cities
for count, province in enumerate(lacuna_provinces):
    print(count, province, lacuna_cities[count])

    # Select Lacuna City <i> from Administrative Boundary Data
    city_gdf = adm_gdf[adm_gdf["ADM3_EN"] == lacuna_cities[count]]

    # Read Provincial 25-Year Flood Hazard Data
    fl_hzrd_fn = f"{INPUT_DIR}/NOAH-Flood-Hazard/25-year/{province}_Flood_25year.shp"
    fl_hzrd_gdf = gpd.read_file(fl_hzrd_fn).to_crs("EPSG:32651")

    # Clip Flood Hazard Data with Lacuna City <i>'s boundary.
    clipped_fl_hzrd_gdf = gpd.clip(fl_hzrd_gdf, city_gdf)

    # Get Intersection of each Hazard with each Baranggay
    overlay_city_fl_hzrd_gdf = gpd.overlay(
        city_gdf, clipped_fl_hzrd_gdf, how="intersection", keep_geom_type=False
    )

    # Calculate the area (inkm2) of the resulting intersecting polygons
    overlay_city_fl_hzrd_gdf["area"] = overlay_city_fl_hzrd_gdf.geometry.area.div(
        1000000
    )

    # Aggregate the area of the intersecting polygons by Baranggay using a Pivot Table
    pivot_city_gdf = overlay_city_fl_hzrd_gdf.pivot_table(
        index="ADM4_EN", columns="Var", values="area", aggfunc=np.sum
    )

    # Rename Columns for Readability. Note that the values are still in km2.
    pivot_city_gdf.columns = pivot_city_column_names_25yr_fl

    # display(pivot_city_gdf)

    # Merge the computed pivot table data to the original city administrative data
    city_gdf = city_gdf.merge(pivot_city_gdf, on="ADM4_EN", how="left")

    # Compute Percent Area out of Baranggay Area
    city_gdf[pivot_city_column_names_25yr_fl] = (
        city_gdf[pivot_city_column_names_25yr_fl].div(
            city_gdf["brgy_total_area_sqkm"], axis=0
        )
        * 100
    )
    # display(city_gdf.iloc[:,10:])

    # Add city data to list of city dataframes
    city_gdf_list.append(city_gdf)

# Concatenate Lists of Lacuna City with Hazard Data Dataframes
fl_25_year_gdf = pd.concat(city_gdf_list)
fl_25_year_gdf.head()

0 Albay Legazpi City


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_total_area_sqkm,brgy_pct_area_flood_hazard_25yr_low,brgy_pct_area_flood_hazard_25yr_med,brgy_pct_area_flood_hazard_25yr_high
0,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 45 - Dita (Bgy. 51),PH050506053,"POLYGON ((579752.343 1458208.276, 580219.811 1...",0.8428,2.035394,0.572573,0.059326
1,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 62 - Homapon (Bgy. 55),PH050506056,"POLYGON ((582204.624 1447546.937, 582206.580 1...",7.0818,NaN,NaN,NaN
2,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 10 - Cabugao,PH050506072,"POLYGON ((579556.773 1452590.900, 579560.114 1...",0.0335,3.272631,1.791045,NaN
3,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 23 - Imperial Court Subd. (Pob.),PH050506020,"POLYGON ((581731.582 1452914.659, 581758.664 1...",0.0621,17.267672,70.615647,5.903528
4,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 40 - Cruzada (Bgy. 52),PH050506051,"POLYGON ((580282.649 1455133.588, 580380.407 1...",1.4442,13.789808,4.550538,0.047622


### 2. Plotting Flood Hazard Maps

In [ ]:
# Create a loop across all Lacuna Cities
for count, province in enumerate(lacuna_provinces):
    # print(count,province, lacuna_cities[count])

    # Select Lacuna City <i> from Administrative Boundary Data
    city_gdf = adm_gdf[adm_gdf["ADM3_EN"] == lacuna_cities[count]]

    # Read Provincial 25-Year Flood Hazard Data
    fl_hzrd_fn = f"{INPUT_DIR}/FloodHazard/25-year/{province}_Flood_25year.shp"
    fl_hzrd_gdf = gpd.read_file(fl_hzrd_fn).to_crs("EPSG:32651")

    # Clip Flood Hazard Data with Lacuna City <i>'s boundary.
    clipped_fl_hzrd_gdf = gpd.clip(fl_hzrd_gdf, city_gdf)

    # Create the figure and axes
    fig, ax = plt.subplots()

    # Plot the Administrative Boundary Basemap (first layer)
    city_gdf.plot(ax=ax, facecolor="none", edgecolor="gray")

    # Crop the visualization to the extent of the first layer
    ax.set_xlim(city_gdf.total_bounds[0], city_gdf.total_bounds[2])
    ax.set_ylim(city_gdf.total_bounds[1], city_gdf.total_bounds[3])
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    # Plot the second layer (flood hazard)
    clipped_fl_hzrd_gdf.plot(
        ax=ax,
        column="Var",
        categorical=True,
        cmap="YlOrRd",
        alpha=0.7,
        legend=True,
        legend_kwds={"loc": "upper center", "bbox_to_anchor": (0.5, -0.05), "ncol": 3},
    )

    # Set Title
    plt.title("25-Year Flood Hazard Map of " + lacuna_cities[count])

    # Save the figure
    # plt.savefig(f'{OUTPUT_DIR}/Plots/{lacuna_cities[count]}_Flood_25_year.png')

    # Show the plot
    plt.show()

## D. 5-year Flood Hazard

### 1. Extracting Hazard Level Percent Area

Compute the percent area of each 5-year flood hazard level per baranggay across the Lacuna Cities. 

In [17]:
# Setup an empty list that will eventually contain all the Lacuna City Dataframes with ammended Flood Hazard Area Data
city_gdf_list = []

# Target Output Columns
pivot_city_column_names_5yr_fl = [
    "brgy_pct_area_flood_hazard_5yr_low",
    "brgy_pct_area_flood_hazard_5yr_med",
    "brgy_pct_area_flood_hazard_5yr_high",
]

# Create a loop across all Lacuna Cities
for count, province in enumerate(lacuna_provinces):
    print(count, province, lacuna_cities[count])

    # Select Lacuna City <i> from Administrative Boundary Data
    city_gdf = adm_gdf[adm_gdf["ADM3_EN"] == lacuna_cities[count]]

    # Read Provincial 5-Year Flood Hazard Data
    fl_hzrd_fn = f"{INPUT_DIR}/NOAH-Flood-Hazard/5-year/{province}_Flood_5year.shp"
    fl_hzrd_gdf = gpd.read_file(fl_hzrd_fn).to_crs("EPSG:32651")

    # Clip Flood Hazard Data with Lacuna City <i>'s boundary.
    clipped_fl_hzrd_gdf = gpd.clip(fl_hzrd_gdf, city_gdf)

    # Get Intersection of each Hazard with each Baranggay
    overlay_city_fl_hzrd_gdf = gpd.overlay(
        city_gdf, clipped_fl_hzrd_gdf, how="intersection", keep_geom_type=False
    )

    # Calculate the area (inkm2) of the resulting intersecting polygons
    overlay_city_fl_hzrd_gdf["area"] = overlay_city_fl_hzrd_gdf.geometry.area.div(
        1000000
    )

    # Aggregate the area of the intersecting polygons by Baranggay using a Pivot Table
    pivot_city_gdf = overlay_city_fl_hzrd_gdf.pivot_table(
        index="ADM4_EN", columns="Var", values="area", aggfunc=np.sum
    )

    # Rename Columns for Readability. Note that the values are still in km2.
    pivot_city_gdf.columns = pivot_city_column_names_5yr_fl

    # display(pivot_city_gdf)

    # Merge the computed pivot table data to the original city administrative data
    city_gdf = city_gdf.merge(pivot_city_gdf, on="ADM4_EN", how="left")

    # Compute Percent Area out of Baranggay Area
    city_gdf[pivot_city_column_names_5yr_fl] = (
        city_gdf[pivot_city_column_names_5yr_fl].div(
            city_gdf["brgy_total_area_sqkm"], axis=0
        )
        * 100
    )
    # display(city_gdf.iloc[:,10:])

    # Add city data to list of city dataframes
    city_gdf_list.append(city_gdf)

# Concatenate Lists of Lacuna City with Hazard Data Dataframes
fl_5_year_gdf = pd.concat(city_gdf_list)
fl_5_year_gdf.head()

0 Albay Legazpi City


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_total_area_sqkm,brgy_pct_area_flood_hazard_5yr_low,brgy_pct_area_flood_hazard_5yr_med,brgy_pct_area_flood_hazard_5yr_high
0,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 45 - Dita (Bgy. 51),PH050506053,"POLYGON ((579752.343 1458208.276, 580219.811 1...",0.8428,1.374969,0.381767,0.035596
1,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 62 - Homapon (Bgy. 55),PH050506056,"POLYGON ((582204.624 1447546.937, 582206.580 1...",7.0818,NaN,NaN,NaN
2,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 10 - Cabugao,PH050506072,"POLYGON ((579556.773 1452590.900, 579560.114 1...",0.0335,2.480850,1.194030,NaN
3,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 23 - Imperial Court Subd. (Pob.),PH050506020,"POLYGON ((581731.582 1452914.659, 581758.664 1...",0.0621,20.903545,51.608041,NaN
4,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 40 - Cruzada (Bgy. 52),PH050506051,"POLYGON ((580282.649 1455133.588, 580380.407 1...",1.4442,9.629987,2.982314,0.040607


### 2. Plotting Flood Hazard Maps

In [ ]:
# Create a loop across all Lacuna Cities
for count, province in enumerate(lacuna_provinces):
    # print(count,province, lacuna_cities[count])

    # Select Lacuna City <i> from Administrative Boundary Data
    city_gdf = adm_gdf[adm_gdf["ADM3_EN"] == lacuna_cities[count]]

    # Read Provincial 5-Year Flood Hazard Data
    fl_hzrd_fn = f"{INPUT_DIR}/FloodHazard/5-year/{province}_Flood_5year.shp"
    fl_hzrd_gdf = gpd.read_file(fl_hzrd_fn).to_crs("EPSG:32651")

    # Clip Flood Hazard Data with Lacuna City <i>'s boundary.
    clipped_fl_hzrd_gdf = gpd.clip(fl_hzrd_gdf, city_gdf)

    # Create the figure and axes
    fig, ax = plt.subplots()

    # Plot the Administrative Boundary Basemap (first layer)
    city_gdf.plot(ax=ax, facecolor="none", edgecolor="gray")

    # Crop the visualization to the extent of the first layer
    ax.set_xlim(city_gdf.total_bounds[0], city_gdf.total_bounds[2])
    ax.set_ylim(city_gdf.total_bounds[1], city_gdf.total_bounds[3])
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    # Plot the second layer (flood hazard)
    clipped_fl_hzrd_gdf.plot(
        ax=ax,
        column="Var",
        categorical=True,
        cmap="YlOrRd",
        alpha=0.7,
        legend=True,
        legend_kwds={"loc": "upper center", "bbox_to_anchor": (0.5, -0.05), "ncol": 3},
    )

    # Set Title
    plt.title("5-Year Flood Hazard Map of " + lacuna_cities[count])

    # Save the figure
    # plt.savefig(f'{OUTPUT_DIR}/Plots/{lacuna_cities[count]}_Flood_5_year.png')

    # Show the plot
    plt.show()

## E. Landslide Hazard

### 1. Extracting Hazard Level Percent Area

Compute the percent area of each landslide hazard level per baranggay across the Lacuna Cities. 

In [18]:
# Setup an empty list that will eventually contain all the Lacuna City Dataframes with ammended Landslide Hazard Area Data
city_gdf_list = []

# Column Names for Readability
pivot_city_column_names_ls = [
    "brgy_pct_area_landslide_hazard_low",
    "brgy_pct_area_landslide_hazard_med",
    "brgy_pct_area_landslide_hazard_high",
]

# Create a loop across all Lacuna Cities
for count, province in enumerate(lacuna_provinces):
    print(count, province, lacuna_cities[count])

    # Select Lacuna City <i> from Administrative Boundary Data
    city_gdf = adm_gdf[adm_gdf["ADM3_EN"] == lacuna_cities[count]]

    # Read Provincial 25-Year Landslide Hazard Data
    ls_hzrd_fn = f"{INPUT_DIR}/NOAH-Landslide-Hazard/Landslide-Hazard/{province}_LandslideHazards.shp"
    ls_hzrd_gdf = gpd.read_file(ls_hzrd_fn).to_crs("EPSG:32651")

    # Clip Landslide Hazard Data with Lacuna City <i>'s boundary.
    clipped_ls_hzrd_gdf = gpd.clip(ls_hzrd_gdf, city_gdf)

    # Get Intersection of each Hazard with each Baranggay
    overlay_city_ls_hzrd_gdf = gpd.overlay(
        city_gdf, clipped_ls_hzrd_gdf, how="intersection", keep_geom_type=False
    )

    # Calculate the area (inkm2) of the resulting intersecting polygons
    overlay_city_ls_hzrd_gdf["area"] = overlay_city_ls_hzrd_gdf.geometry.area.div(
        1000000
    )

    # Aggregate the area of the intersecting polygons by Baranggay using a Pivot Table
    pivot_city_gdf = overlay_city_ls_hzrd_gdf.pivot_table(
        index="ADM4_EN", columns="LH", values="area", aggfunc=np.sum
    )

    # Rename Columns for Readability. Note that the values are still in km2.
    pivot_city_gdf.columns = pivot_city_column_names_ls

    # display(pivot_city_gdf)

    # Merge the computed pivot table data to the original city administrative data
    city_gdf = city_gdf.merge(pivot_city_gdf, on="ADM4_EN", how="left")

    # Compute Percent Area out of Baranggay Area
    city_gdf[pivot_city_column_names_ls] = (
        city_gdf[pivot_city_column_names_ls].div(
            city_gdf["brgy_total_area_sqkm"], axis=0
        )
        * 100
    )
    # display(city_gdf.iloc[:,10:])

    # Add city data to list of city dataframes
    city_gdf_list.append(city_gdf)

# Concatenate Lists of Lacuna City with Hazard Data Dataframes
ls_gdf = pd.concat(city_gdf_list)
ls_gdf.head()

0 Albay Legazpi City


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_total_area_sqkm,brgy_pct_area_landslide_hazard_low,brgy_pct_area_landslide_hazard_med,brgy_pct_area_landslide_hazard_high
0,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 45 - Dita (Bgy. 51),PH050506053,"POLYGON ((579752.343 1458208.276, 580219.811 1...",0.8428,NaN,NaN,NaN
1,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 62 - Homapon (Bgy. 55),PH050506056,"POLYGON ((582204.624 1447546.937, 582206.580 1...",7.0818,10.134472,7.927245,1.919271
2,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 10 - Cabugao,PH050506072,"POLYGON ((579556.773 1452590.900, 579560.114 1...",0.0335,NaN,NaN,NaN
3,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 23 - Imperial Court Subd. (Pob.),PH050506020,"POLYGON ((581731.582 1452914.659, 581758.664 1...",0.0621,NaN,NaN,NaN
4,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 40 - Cruzada (Bgy. 52),PH050506051,"POLYGON ((580282.649 1455133.588, 580380.407 1...",1.4442,NaN,NaN,NaN


### 2. Plotting Landslide Hazard Maps

In [ ]:
# Create a loop across all Lacuna Cities
for count, province in enumerate(lacuna_provinces):
    # print(count,province, lacuna_cities[count])

    # Select Lacuna City <i> from Administrative Boundary Data
    city_gdf = adm_gdf[adm_gdf["ADM3_EN"] == lacuna_cities[count]]

    # Read Provincial Landslide Hazard Data
    ls_hzrd_fn = f"{INPUT_DIR}/NOAH-Landslide-Hazard/Landslide-Hazard/{province}_LandslideHazards.shp"
    ls_hzrd_gdf = gpd.read_file(ls_hzrd_fn).to_crs("EPSG:32651")

    # Clip Landslide Hazard Data with Lacuna City <i>'s boundary.
    clipped_ls_hzrd_gdf = gpd.clip(ls_hzrd_gdf, city_gdf)

    # Create the figure and axes
    fig, ax = plt.subplots()

    # Plot the Administrative Boundary Basemap (first layer)
    city_gdf.plot(ax=ax, facecolor="none", edgecolor="gray")

    # Crop the visualization to the extent of the first layer
    ax.set_xlim(city_gdf.total_bounds[0], city_gdf.total_bounds[2])
    ax.set_ylim(city_gdf.total_bounds[1], city_gdf.total_bounds[3])
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    # Plot the second layer (Landslide hazard)
    clipped_ls_hzrd_gdf.plot(
        ax=ax,
        column="LH",
        categorical=True,
        cmap="YlOrRd",
        alpha=0.7,
        legend=True,
        legend_kwds={"loc": "upper center", "bbox_to_anchor": (0.5, -0.05), "ncol": 3},
    )

    # Set Title
    plt.title("Landslide Hazard Map of " + lacuna_cities[count])

    # Save the figure
    # plt.savefig(f'{OUTPUT_DIR}/Plots/{lacuna_cities[count]}_Landslide_5_year.png')

    # Show the plot
    plt.show()

# III. Merging all Hazard Data

Merge all hazard percent area data from NOAH Flood Hazard (100yr, 25yr, 5yr) and Landslide Hazard Data

In [19]:
# Merge Hazard Percent Area Data (from 100-year, 25-year, 5-year Flood Hazard to Landslide Hazazrd)
fl_100_25_gdf = pd.merge(
    fl_100_year_gdf,
    fl_25_year_gdf[["ADM4_PCODE"] + pivot_city_column_names_25yr_fl],
    on="ADM4_PCODE",
    how="left",
)
fl_100_25_5_gdf = pd.merge(
    fl_100_25_gdf,
    fl_5_year_gdf[["ADM4_PCODE"] + pivot_city_column_names_5yr_fl],
    on="ADM4_PCODE",
    how="left",
)
all_hazards_gdf = pd.merge(
    fl_100_25_5_gdf,
    ls_gdf[["ADM4_PCODE"] + pivot_city_column_names_ls],
    on="ADM4_PCODE",
    how="left",
)

# Fill NaN Hazard Percent Area Data values
all_hazards_gdf.iloc[:, 10:] = all_hazards_gdf.iloc[:, 10:].fillna(0).round(4)
all_hazards_gdf.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_total_area_sqkm,...,brgy_pct_area_flood_hazard_100yr_high,brgy_pct_area_flood_hazard_25yr_low,brgy_pct_area_flood_hazard_25yr_med,brgy_pct_area_flood_hazard_25yr_high,brgy_pct_area_flood_hazard_5yr_low,brgy_pct_area_flood_hazard_5yr_med,brgy_pct_area_flood_hazard_5yr_high,brgy_pct_area_landslide_hazard_low,brgy_pct_area_landslide_hazard_med,brgy_pct_area_landslide_hazard_high
0,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 45 - Dita (Bgy. 51),PH050506053,"POLYGON ((579752.343 1458208.276, 580219.811 1...",0.8428,...,0.0907,2.0354,0.5726,0.0593,1.3750,0.3818,0.0356,0.0000,0.0000,0.0000
1,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 62 - Homapon (Bgy. 55),PH050506056,"POLYGON ((582204.624 1447546.937, 582206.580 1...",7.0818,...,6.5147,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,10.1345,7.9272,1.9193
2,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 10 - Cabugao,PH050506072,"POLYGON ((579556.773 1452590.900, 579560.114 1...",0.0335,...,0.0000,3.2726,1.7910,0.0000,2.4808,1.1940,0.0000,0.0000,0.0000,0.0000
3,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 23 - Imperial Court Subd. (Pob.),PH050506020,"POLYGON ((581731.582 1452914.659, 581758.664 1...",0.0621,...,22.8379,17.2677,70.6156,5.9035,20.9035,51.6080,0.0000,0.0000,0.0000,0.0000
4,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,Bgy. 40 - Cruzada (Bgy. 52),PH050506051,"POLYGON ((580282.649 1455133.588, 580380.407 1...",1.4442,...,0.1185,13.7898,4.5505,0.0476,9.6300,2.9823,0.0406,0.0000,0.0000,0.0000


# IV. Export 

In [20]:
# Export GEOJSON
all_hazards_gdf.to_crs("EPSG:4326").to_file(
    "{OUTPUT_DIR}/hz_proportion.json", driver="GeoJSON"
)

/home/cczablan1/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [ ]:
# Export CSV
all_hazards_gdf.drop(["geometry"], axis=1).to_csv(
    "{OUTPUT_DIR}/hz_proportion.csv", encoding="utf-8", index=False
)